## 1B.- A transaction class to send and receive money and test it.


In [2]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00


In [3]:
#1B.- A transaction class to send and receive money and test it.
import Crypto
import binascii

import datetime
import collections

from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA

In [4]:
class Client:
    def __init__(self):
        # Creating random number for key
        random = Crypto.Random.new().read
        # Creating new public key and private key
        self._private_key = RSA.generate(1024, random)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)

    @property
    def identity(self):
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

In [5]:
class Transaction:
    def __init__(self, sender, receiver, value):
        self.sender = sender
        self.receiver = receiver
        self.value = value
        self.time = datetime.datetime.now()

    def to_dict(self):
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity

        return collections.OrderedDict({
            'sender': identity,
            'receiver': self.receiver,
            'value': self.value,
            'time': self.time
        })

    def sign_transaction(self):
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')

In [6]:
Ninad = Client()
print("-"*50)
print("Ninad Key")
print(Ninad.identity)

KS = Client()
print("-"*50)
print("KS Key")
print(KS.identity)

t = Transaction(Ninad, KS.identity, 10.0)
print("-"*50)
print("Transaction Sign")
signature = t.sign_transaction()
print(signature)
print("-"*50)

--------------------------------------------------
Ninad Key
30819f300d06092a864886f70d010101050003818d0030818902818100a99ad7dbf3cbfbb0d340ae1d091387896b0ef9449e032569819ace751b04c482540dec6ab8e7b8179295347aa2ecf4eaddb88c079f7427cf229d0f0015acf960c402b18ee338a1b911899d37aa884abca06743dcc06878ad2b787da153f038f2f25b54a214f67ea121052de5637b34a89d553a6708d1575a081bc4dd411444b90203010001
--------------------------------------------------
KS Key
30819f300d06092a864886f70d010101050003818d0030818902818100b03818bf884264881b44027dd3e654fdd258339df2d3040ad2122d0e1329830cc625b29f9c67b28fb4f244244e7f74deb23b1b1901c4c726368aadd1b7b9a86211ad6e3a6514b8d99bc93749e4f4c75db63a9fbcb15fff07dde42f4e618d456da1d816aecb8aa5b2093ec336cad7037e10ffd025058df7f8f80bc1d1cf6197150203010001
--------------------------------------------------
Transaction Sign
a7a209a226c7df822f436dae08b64dab0dcc0c744ef7ff198868ee005b3e570969e4cdb817ca36c1d69b999e308c1ae6762755df2101c1ee4de31f29ee4a98a635ea8cca3effecf13061087495f82c0cbba